In [1]:
import dask.dataframe as dd
import pandas as pd
from tqdm import tqdm

# Função para processar em pandas
def import_prices(df):
    # Ordenar o DataFrame pelos campos necessários
    df = df.sort_values(by=['item', 'store_code', 'sell_price', 'yearweek'])

    # Preencher os valores NaN com o valor anterior
    df['yearweek'] = df['yearweek'].ffill()

    # Converter a coluna yearweek para inteiro
    df['yearweek'] = df['yearweek'].astype(int)

    # Função para garantir a unicidade de yearweek, incrementando se necessário
    def increment_yearweek(df):
        last_yearweek = None
        for i, row in df.iterrows():
            if last_yearweek is not None and row['yearweek'] <= last_yearweek:
                last_yearweek += 1
                df.at[i, 'yearweek'] = int(last_yearweek)
            else:
                last_yearweek = row['yearweek']
        return df

    # Aplicar a função para incrementar yearweek dentro de cada grupo de item, store_code, e sell_price
    df = df.groupby(['item', 'store_code', 'sell_price']).apply(increment_yearweek).reset_index(drop=True)

    return df

# Carregar o arquivo item_prices.csv com Dask
item_prices = dd.read_csv('../data/item_prices.csv', dtype={'yearweek': 'float64'})

# Aplicar a função de pandas a cada partição do Dask DataFrame
df_filled_dask = item_prices.map_partitions(import_prices)

# Salvar como múltiplos arquivos Parquet
df_filled_dask.to_parquet('../data/item_prices_filled_parquet/', write_index=False)


/tmp/ipykernel_61314/1630260444.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['item', 'store_code', 'sell_price']).apply(increment_yearweek).reset_index(drop=True)
